In [ ]:
# default_exp preprocessing.clean

# Clean

> Functions to split the raw EHR dataset, clean and save for further processing & vocab creation.

In [ ]:
#hide
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2

In [ ]:
#export
from lemonade.setup import *
from fastai.imports import *

In [ ]:
#hide
from nbdev.showdoc import *

## Split 

Once dataset is assembled, the folder will look as follows .. 

In [ ]:
os.listdir(f'{PATH_1K}/raw_original')

['patients.csv',
 'observations.csv',
 'allergies.csv',
 'careplans.csv',
 'medications.csv',
 'imaging_studies.csv',
 'procedures.csv',
 'conditions.csv',
 'encounters.csv',
 'immunizations.csv']

In [ ]:
#export
def read_raw_ehrdata(path, csv_names = FILENAMES):
    '''Read raw EHR data'''
    dfs = [pd.read_csv(f'{path}/{fname}.csv', low_memory=False) for fname in csv_names]
    return dfs    

In [ ]:
dfs = read_raw_ehrdata(f'{PATH_1K}/raw_original')

In [ ]:
patients, observations, allergies, careplans, medications, imaging_studies, procedures, conditions, immunizations = dfs

In [ ]:
#export
def split_patients(patients, valid_pct=0.2, test_pct=0.2, random_state=1234):
    '''Split the patients dataframe'''
    train_pct = 1 - (valid_pct + test_pct)
    print(f'Splits:: train: {train_pct}, valid: {valid_pct}, test: {test_pct}')
    patients = patients.sample(frac=1, random_state=random_state).reset_index(drop=True)
    return np.split(patients, [int(train_pct*len(patients)), int((train_pct+valid_pct)*len(patients))])

In [ ]:
train, valid, test = split_patients(patients, .2,.1)

Splits:: train: 0.7, valid: 0.2, test: 0.1


In [ ]:
len(patients), len(train), len(valid), len(test)

(1108, 775, 222, 111)

In [ ]:
assert len(patients) == len(train)+len(valid)+len(test)

In [ ]:
#export
def split_ehr_dataset(path, valid_pct=0.2, test_pct=0.2, random_state=1234):
    '''Split EHR dataset into train, valid, test and save'''

    train_dfs, valid_dfs, test_dfs = [],[],[]
    
    dfs = read_raw_ehrdata(f'{path}/raw_original')
    train_pt, valid_pt, test_pt = split_patients(dfs[0], valid_pct, test_pct, random_state)
    train_dfs.append(train_pt)
    valid_dfs.append(valid_pt)
    test_dfs.append(test_pt)
    print(f'Split {FILENAMES[0]} into:: Train: {len(train_pt)}, Valid: {len(valid_pt)}, Test: {len(test_pt)} -- Total before split: {len(dfs[0])}')
    
    for df, name in zip(dfs[1:], FILENAMES[1:]):
        df = df.set_index('PATIENT')
        df_train = df.loc[df.index.intersection(train_pt['ID']).unique()]
        df_valid = df.loc[df.index.intersection(valid_pt['ID']).unique()]
        df_test = df.loc[df.index.intersection(test_pt['ID']).unique()]
        assert len(df) == len(df_train)+len(df_valid)+len(df_test),f'Split failed {name}: {len(df)} != {len(df_train)}+{len(df_valid)}+{len(df_test)}'
        train_dfs.append(df_train.reset_index())
        valid_dfs.append(df_valid.reset_index())
        test_dfs.append(df_test.reset_index())

    
    for split in ['train', 'valid', 'test']:
        d = Path(f'{path}/raw_split/{split}')
        d.mkdir(parents=True, exist_ok=True)
        
        if split == 'train':
            for df, name in zip(train_dfs, FILENAMES):
                df.to_csv(f'{d}/{name}.csv', index=False)
            print(f'Saved train data to {d}')
        
        if split == 'valid':
            for df, name in zip(valid_dfs, FILENAMES):
                df.to_csv(f'{d}/{name}.csv', index=False)
            print(f'Saved valid data to {d}')
    
        if split == 'test':
            for df, name in zip(test_dfs, FILENAMES):
                df.to_csv(f'{d}/{name}.csv', index=False)
            print(f'Saved test data to {d}')

In [ ]:
split_ehr_dataset(PATH_1K) #will use default values for split percents

Splits:: train: 0.6, valid: 0.2, test: 0.2
Split patients into:: Train: 664, Valid: 222, Test: 222 -- Total before split: 1108
Saved train data to /home/vinod/code/datasets/synthea/1K/raw_split/train
Saved valid data to /home/vinod/code/datasets/synthea/1K/raw_split/valid
Saved test data to /home/vinod/code/datasets/synthea/1K/raw_split/test


## Clean

In [ ]:
#export
def cleanup_pts(pts, is_train, today=None):
    '''Clean patients df'''
    pts.drop(columns=['SSN','DRIVERS','PASSPORT','PREFIX','FIRST','LAST','SUFFIX','MAIDEN','ADDRESS'], inplace=True)
    pts.rename(str.lower, axis='columns', inplace=True)
    pts.rename(columns={"id":"patient"}, inplace=True)
    pts = pts.astype({'birthdate':'datetime64'}) 
    pts['zip'] = pts['zip'].fillna(0.0).astype(int)    
    if today == None: today = pd.Timestamp.today()
    else            : today = pd.to_datetime(today)
    pts['age_now_days'] = pts['birthdate'].apply(lambda bday: (today-bday).days)
    
    pts.set_index('patient', inplace=True)
    pt_demographics = pts.loc[:,['birthdate', 'marital', 'race', 'ethnicity', 'gender', 'birthplace', 'city', 'state', 'zip', 'age_now_days']] 
    patients = pts.loc[:,['birthdate']]
    if is_train: pt_codes = pt_demographics.reset_index(drop=True)
    
    return [patients, pt_demographics, pt_codes] if is_train else [patients, pt_demographics]

`patients` data frame looks like this before cleanup.. 

In [ ]:
patients.head()

,ID,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,ZIP
0,d16cfd6a-d9c3-4754-a9f2-e8ce0b7ec40e,1990-05-05,NaN,999-45-9813,S99921748,X74952757X,Mrs.,Nanci249,Hettinger594,NaN,Jones311,M,white,english,F,Bourne,540 Nikolaus Meadow,Billerica,Massachusetts,1821.0
1,92cb2e9a-1a29-450d-8a36-0242cf3c5355,1983-05-28,NaN,999-10-2955,S99983211,X67532687X,Mr.,Don899,Mitchell808,NaN,NaN,M,white,irish,M,Worcester,512 Pfeffer Byway,Shutesbury,Massachusetts,1072.0
2,116f1680-4fc0-4b2d-a59e-0e62231efe12,2011-10-23,NaN,999-97-3322,NaN,NaN,NaN,Belia417,Zieme486,NaN,NaN,NaN,white,irish,F,Braintree Town,891 Graham Harbor,Gloucester,Massachusetts,1930.0
3,ffc93b3e-3fb3-4a6d-a53d-61de18ef4c23,1969-02-12,NaN,999-28-4068,S99952230,X32221027X,Mr.,Rhett759,Mosciski958,NaN,NaN,M,white,polish,M,Melrose,609 Eichmann Avenue,Woburn,Massachusetts,1801.0
4,3189d8e4-653a-47c5-a700-941ebf2d02c6,1969-01-23,NaN,999-64-1895,S99914844,X20718523X,Mrs.,Augustine565,Rohan584,NaN,Cremin516,M,white,american,F,Upton,931 Watsica Lock,Worcester,Massachusetts,1545.0


In [ ]:
pt_data = cleanup_pts(patients, is_train=True, today=SYNTHEA_DATAGEN_DATES['1K'])
patients, pt_demo, pt_codes = pt_data[0], pt_data[1], pt_data[2]

Our cleanup function produces the following 3 dfs - `patients`, `pt_demographics`, `pt_codes`

In [ ]:
for df in pt_data:
    display(df.head())

,birthdate
patient,
d16cfd6a-d9c3-4754-a9f2-e8ce0b7ec40e,1990-05-05
92cb2e9a-1a29-450d-8a36-0242cf3c5355,1983-05-28
116f1680-4fc0-4b2d-a59e-0e62231efe12,2011-10-23
ffc93b3e-3fb3-4a6d-a53d-61de18ef4c23,1969-02-12
3189d8e4-653a-47c5-a700-941ebf2d02c6,1969-01-23


,birthdate,marital,race,ethnicity,gender,birthplace,city,state,zip,age_now_days
patient,,,,,,,,,,
d16cfd6a-d9c3-4754-a9f2-e8ce0b7ec40e,1990-05-05,M,white,english,F,Bourne,Billerica,Massachusetts,1821,10422
92cb2e9a-1a29-450d-8a36-0242cf3c5355,1983-05-28,M,white,irish,M,Worcester,Shutesbury,Massachusetts,1072,12956
116f1680-4fc0-4b2d-a59e-0e62231efe12,2011-10-23,NaN,white,irish,F,Braintree Town,Gloucester,Massachusetts,1930,2581
ffc93b3e-3fb3-4a6d-a53d-61de18ef4c23,1969-02-12,M,white,polish,M,Melrose,Woburn,Massachusetts,1801,18174
3189d8e4-653a-47c5-a700-941ebf2d02c6,1969-01-23,M,white,american,F,Upton,Worcester,Massachusetts,1545,18194


,birthdate,marital,race,ethnicity,gender,birthplace,city,state,zip,age_now_days
0,1990-05-05,M,white,english,F,Bourne,Billerica,Massachusetts,1821,10422
1,1983-05-28,M,white,irish,M,Worcester,Shutesbury,Massachusetts,1072,12956
2,2011-10-23,NaN,white,irish,F,Braintree Town,Gloucester,Massachusetts,1930,2581
3,1969-02-12,M,white,polish,M,Melrose,Woburn,Massachusetts,1801,18174
4,1969-01-23,M,white,american,F,Upton,Worcester,Massachusetts,1545,18194


**The case for keeping a record of the data generation date**

Also note the difference in `age_now` if it were calculated based on default (`pd.Timestamp.today()`) vs `SYNTHEA_DATAGEN_DATES['1K']` which is the data generation date for this 1K dataset.  

In [ ]:
# age_now -- today vs run_date
(pd.to_datetime(pd.Timestamp.today()) - patients.iloc[2])[0].days, (pd.to_datetime(SYNTHEA_DATAGEN_DATES['1K']) - patients.iloc[2])[0].days

(3425, 2581)

In [ ]:
#export
def cleanup_obs(obs, is_train):
    '''Clean observations df'''
    obs.UNITS.fillna('xxxnan', inplace=True)
    obs.dropna(subset=['VALUE'], inplace=True)
    obs.rename(columns={"DATE":"date", "PATIENT":"patient", "CODE":"orig_code", \
                                 "DESCRIPTION":"desc", "VALUE":"value", "UNITS":"units", "TYPE":"type"}, inplace=True)
    obs['code'] = obs['orig_code'].str.cat(obs[['value', 'units', 'type']].astype(str), sep='||')

    if is_train: obs_codes = obs.loc[:, ['orig_code', 'desc', 'value', 'units', 'type']]
    
    obs = obs.loc[:, ['patient', 'date', 'code']]
    obs = obs.astype({'date':'datetime64'})
    obs.set_index('patient', inplace=True)
    
    return [obs, obs_codes] if is_train else [obs]

- Drops rows with null in the `VALUE` column
- Creates a new `code` column with a concatenation of `code`, `value`, `units` and `type`
    - so that we can use the following logic during vocab creation for observations (further detailed in the vocab documentation)

**For `numeric`**
```
for 'numeric'
    get unique 'codes'
    for each unique code
        get unique 'units'
            for each unique unit
                bucketize 'values'
                create vocab entry for each 'bucket' -- code||value_bucket||units
```
**For `text`**
```
for 'text'
    get unique 'codes'
    for each unique code
        get unique 'units' #this will be null
            for each unique unit
                get unique 'values'
                create vocab entry for each -- code||value||units
```

'observations' df before cleanup ..

In [ ]:
observations.head()

,DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION,VALUE,UNITS,TYPE
0,1990-05-05,d16cfd6a-d9c3-4754-a9f2-e8ce0b7ec40e,62094db5-574e-4102-9488-ca3906ea85ff,8302-2,Body Height,58.3,cm,numeric
1,1990-05-05,d16cfd6a-d9c3-4754-a9f2-e8ce0b7ec40e,62094db5-574e-4102-9488-ca3906ea85ff,72514-3,Pain severity - 0-10 verbal numeric rating [Sc...,1.6,{score},numeric
2,1990-05-05,d16cfd6a-d9c3-4754-a9f2-e8ce0b7ec40e,62094db5-574e-4102-9488-ca3906ea85ff,29463-7,Body Weight,3.7,kg,numeric
3,1990-05-05,d16cfd6a-d9c3-4754-a9f2-e8ce0b7ec40e,62094db5-574e-4102-9488-ca3906ea85ff,6690-2,Leukocytes [#/volume] in Blood by Automated count,7.9,10*3/uL,numeric
4,1990-05-05,d16cfd6a-d9c3-4754-a9f2-e8ce0b7ec40e,62094db5-574e-4102-9488-ca3906ea85ff,789-8,Erythrocytes [#/volume] in Blood by Automated ...,4.5,10*6/uL,numeric


In [ ]:
obs_data = cleanup_obs(observations, is_train=True)
observations, obs_codes = obs_data[0], obs_data[1]

after cleanup..

In [ ]:
for df in obs_data:
    display(df.head())

,date,code
patient,,
d16cfd6a-d9c3-4754-a9f2-e8ce0b7ec40e,1990-05-05,8302-2||58.3||cm||numeric
d16cfd6a-d9c3-4754-a9f2-e8ce0b7ec40e,1990-05-05,72514-3||1.6||{score}||numeric
d16cfd6a-d9c3-4754-a9f2-e8ce0b7ec40e,1990-05-05,29463-7||3.7||kg||numeric
d16cfd6a-d9c3-4754-a9f2-e8ce0b7ec40e,1990-05-05,6690-2||7.9||10*3/uL||numeric
d16cfd6a-d9c3-4754-a9f2-e8ce0b7ec40e,1990-05-05,789-8||4.5||10*6/uL||numeric


,orig_code,desc,value,units,type
0,8302-2,Body Height,58.3,cm,numeric
1,72514-3,Pain severity - 0-10 verbal numeric rating [Sc...,1.6,{score},numeric
2,29463-7,Body Weight,3.7,kg,numeric
3,6690-2,Leukocytes [#/volume] in Blood by Automated count,7.9,10*3/uL,numeric
4,789-8,Erythrocytes [#/volume] in Blood by Automated ...,4.5,10*6/uL,numeric


In [ ]:
#export
def cleanup_algs(allergies, is_train):
    '''Clean allergies df'''
    allergies.drop(columns=['ENCOUNTER'], inplace=True)
    
    stops = pd.DataFrame(allergies.loc[allergies['STOP'].notnull(),:])
    allergies['CODE'] = allergies['CODE'].apply(lambda x: f'{str(x)}||START')
    stops['CODE'] = stops['CODE'].apply(lambda x: f'{str(x)}||STOP')
    allergies.drop(columns=['STOP'], inplace=True)
    stops.drop(columns=['START'], inplace=True)
    allergies.rename(columns={"START":"date", "PATIENT":"patient", "CODE":"code", "DESCRIPTION":"desc"}, inplace=True)
    stops.rename(columns={"STOP":"date", "PATIENT":"patient", "CODE":"code", "DESCRIPTION":"desc"}, inplace=True)
    allergies = allergies.append(stops, ignore_index=True)
    
    if is_train: alg_codes = allergies.loc[:, ['code', 'desc']]
        
    allergies.drop(columns=['desc'], inplace=True)
    allergies = allergies.astype({'date':'datetime64'})
    allergies.set_index('patient', inplace=True)
    return [allergies, alg_codes] if is_train else [allergies]

`allergies` have a start and stop date in the same row indicating when an allergy (indicated by its code) started and stopped (or not) for a patient. <br>
So in the cleanup, we flatten that out, meaning create new rows for stop dates. <br>
The dataframe looks as follows before cleanup..

In [ ]:
allergies.head()

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,1956-07-24,NaN,2a45e46d-b4d0-4959-b07d-fb57b25e9f1d,91aff420-4326-42f2-abbc-bb26a5d8ae83,419474003,Allergy to mould
1,1956-07-24,NaN,2a45e46d-b4d0-4959-b07d-fb57b25e9f1d,91aff420-4326-42f2-abbc-bb26a5d8ae83,232350006,House dust mite allergy
2,1956-07-24,1973-01-13,2a45e46d-b4d0-4959-b07d-fb57b25e9f1d,91aff420-4326-42f2-abbc-bb26a5d8ae83,714035009,Allergy to soya
3,1971-05-13,NaN,1cdff491-7e96-4411-acee-d0f080423bb1,b3b00299-8e7a-4591-ae4a-e097dc6e7073,424213003,Allergy to bee venom
4,1971-05-13,NaN,1cdff491-7e96-4411-acee-d0f080423bb1,b3b00299-8e7a-4591-ae4a-e097dc6e7073,419474003,Allergy to mould


In [ ]:
alg_data = cleanup_algs(allergies, is_train=True)
allergies, alg_codes = alg_data[0], alg_data[1]

Resulting in the following output after cleanup.. 

In [ ]:
for df in alg_data:
    display(df.head(3))
    display(df.tail(3))

,date,code
patient,,
2a45e46d-b4d0-4959-b07d-fb57b25e9f1d,1956-07-24,419474003||START
2a45e46d-b4d0-4959-b07d-fb57b25e9f1d,1956-07-24,232350006||START
2a45e46d-b4d0-4959-b07d-fb57b25e9f1d,1956-07-24,714035009||START


,date,code
patient,,
ace46042-749e-44ba-9c7c-92cf42c00eb1,2013-01-23,714035009||STOP
ace46042-749e-44ba-9c7c-92cf42c00eb1,2013-01-23,420174000||STOP
6e57d31b-8e6b-4007-8de1-431ae86a0f12,1993-07-10,419474003||STOP


,code,desc
0,419474003||START,Allergy to mould
1,232350006||START,House dust mite allergy
2,714035009||START,Allergy to soya


,code,desc
846,714035009||STOP,Allergy to soya
847,420174000||STOP,Allergy to wheat
848,419474003||STOP,Allergy to mould


In [ ]:
#export
def cleanup_crpls(careplans, is_train):
    '''Clean careplans df'''
    careplans.drop(columns=['ID', 'ENCOUNTER', 'REASONCODE', 'REASONDESCRIPTION'], inplace=True)
    
    stops = pd.DataFrame(careplans.loc[careplans['STOP'].notnull(),:])
    careplans['CODE'] = careplans['CODE'].apply(lambda x: f'{str(x)}||START')
    stops['CODE'] = stops['CODE'].apply(lambda x: f'{str(x)}||STOP')
    careplans.drop(columns=['STOP'], inplace=True)
    stops.drop(columns=['START'], inplace=True)
    careplans.rename(columns={"START":"date", "PATIENT":"patient", "CODE":"code", "DESCRIPTION":"desc"}, inplace=True)
    stops.rename(columns={"STOP":"date", "PATIENT":"patient", "CODE":"code", "DESCRIPTION":"desc"}, inplace=True)
    careplans = careplans.append(stops, ignore_index=True)
    
    if is_train: crpl_codes = careplans.loc[:, ['code', 'desc']]

    careplans.drop(columns=['desc'], inplace=True)
    careplans = careplans.astype({'date':'datetime64'})
    careplans.set_index('patient', inplace=True)
    return [careplans, crpl_codes] if is_train else [careplans]

In [ ]:
careplans.head()

,ID,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,REASONCODE,REASONDESCRIPTION
0,991acfe9-0f58-4a3e-9af6-f5dcc2c8e1af,2002-12-28,2003-01-18,d16cfd6a-d9c3-4754-a9f2-e8ce0b7ec40e,60717571-4b7b-49f0-8bee-cb99cef3eeeb,225358003,Wound care,283371005.0,Laceration of forearm
1,7b021411-19c2-4a72-8f5b-ae8a63e3bb04,1990-12-19,1992-01-04,92cb2e9a-1a29-450d-8a36-0242cf3c5355,a1363162-be40-40c3-b8a6-ff03892220eb,53950000,Respiratory therapy,10509002.0,Acute bronchitis (disorder)
2,dab0ef53-6e02-4487-9e58-43d6eb847908,2009-07-18,NaN,d16cfd6a-d9c3-4754-a9f2-e8ce0b7ec40e,a143d31d-ca90-47d3-b3aa-be4443913807,134435003,Routine antenatal care,72892002.0,Normal pregnancy
3,2ec6bac2-eee4-4e1a-bafd-e501b9efe17b,2010-03-22,2010-04-05,d16cfd6a-d9c3-4754-a9f2-e8ce0b7ec40e,be9f572c-512f-4e62-b9c8-a190b9a5d5f8,53950000,Respiratory therapy,10509002.0,Acute bronchitis (disorder)
4,b33af643-ad5d-45a4-81b2-3225cd8335dd,1997-06-13,1997-07-11,92cb2e9a-1a29-450d-8a36-0242cf3c5355,f5ed9a41-1faa-44f9-9f0b-75666d5244b7,91251008,Physical therapy procedure,44465007.0,Sprain of ankle


In [ ]:
crpl_data = cleanup_crpls(careplans, is_train=True)
careplans, crpl_codes = crpl_data[0], crpl_data[1]

In [ ]:
for df in crpl_data:
    display(df.head(3))
    display(df.tail(3))

,date,code
patient,,
d16cfd6a-d9c3-4754-a9f2-e8ce0b7ec40e,2002-12-28,225358003||START
92cb2e9a-1a29-450d-8a36-0242cf3c5355,1990-12-19,53950000||START
d16cfd6a-d9c3-4754-a9f2-e8ce0b7ec40e,2009-07-18,134435003||START


,date,code
patient,,
6e57d31b-8e6b-4007-8de1-431ae86a0f12,2003-12-21,385691007||STOP
6e57d31b-8e6b-4007-8de1-431ae86a0f12,2007-02-24,225358003||STOP
6e57d31b-8e6b-4007-8de1-431ae86a0f12,2008-06-07,53950000||STOP


,code,desc
0,225358003||START,Wound care
1,53950000||START,Respiratory therapy
2,134435003||START,Routine antenatal care


,code,desc
16585,385691007||STOP,Fracture care
16586,225358003||STOP,Wound care
16587,53950000||STOP,Respiratory therapy


In [ ]:
#export
def cleanup_meds(medications, is_train):
    '''Clean `medications` df'''
    medications.drop(columns=['ENCOUNTER', 'COST', 'DISPENSES', 'TOTALCOST', 'REASONCODE', 'REASONDESCRIPTION'], inplace=True)
    stops = pd.DataFrame(medications.loc[medications['STOP'].notnull(),:])
    medications['CODE'] = medications['CODE'].apply(lambda x: f'{str(x)}||START')
    stops['CODE'] = stops['CODE'].apply(lambda x: f'{str(x)}||STOP')
    medications.drop(columns=['STOP'], inplace=True)
    stops.drop(columns=['START'], inplace=True)
    medications.rename(columns={"START":"date", "PATIENT":"patient", "CODE":"code", "DESCRIPTION":"desc"}, inplace=True)
    stops.rename(columns={"STOP":"date", "PATIENT":"patient", "CODE":"code", "DESCRIPTION":"desc"}, inplace=True)
    medications = medications.append(stops, ignore_index=True)
    
    if is_train: med_codes = medications.loc[:, ['code', 'desc']]

    medications.drop(columns=['desc'], inplace=True)
    medications = medications.astype({'date':'datetime64'})
    medications.set_index('patient', inplace=True)
    return [medications, med_codes] if is_train else [medications]

In [ ]:
medications.head()

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,COST,DISPENSES,TOTALCOST,REASONCODE,REASONDESCRIPTION
0,1990-08-03,1990-08-17,d16cfd6a-d9c3-4754-a9f2-e8ce0b7ec40e,f46ed016-7cf8-4991-81f5-dad3d9f1b33e,198405,Ibuprofen 100 MG Oral Tablet,7.46,1,7.46,NaN,NaN
1,1994-08-23,1994-09-06,d16cfd6a-d9c3-4754-a9f2-e8ce0b7ec40e,9307c2e7-c41d-4634-9a8f-9dd317fd2acc,308182,Amoxicillin 250 MG Oral Capsule,7.26,1,7.26,NaN,NaN
2,1994-08-23,1994-09-06,d16cfd6a-d9c3-4754-a9f2-e8ce0b7ec40e,9307c2e7-c41d-4634-9a8f-9dd317fd2acc,198405,Ibuprofen 100 MG Oral Tablet,4.40,1,4.40,NaN,NaN
3,2000-02-06,2000-02-13,d16cfd6a-d9c3-4754-a9f2-e8ce0b7ec40e,6906c7bc-cbf2-41bd-a10d-4112248a6bac,834061,Penicillin V Potassium 250 MG Oral Tablet,5.95,1,5.95,43878008.0,Streptococcal sore throat (disorder)
4,2002-12-28,2003-01-18,d16cfd6a-d9c3-4754-a9f2-e8ce0b7ec40e,60717571-4b7b-49f0-8bee-cb99cef3eeeb,313782,Acetaminophen 325 MG Oral Tablet,5.36,1,5.36,NaN,NaN


In [ ]:
med_data = cleanup_meds(medications, is_train=True)
medications, med_codes = med_data[0], med_data[1]

In [ ]:
for df in med_data:
    display(df.head(3))
    display(df.tail(3))

,date,code
patient,,
d16cfd6a-d9c3-4754-a9f2-e8ce0b7ec40e,1990-08-03,198405||START
d16cfd6a-d9c3-4754-a9f2-e8ce0b7ec40e,1994-08-23,308182||START
d16cfd6a-d9c3-4754-a9f2-e8ce0b7ec40e,1994-08-23,198405||START


,date,code
patient,,
6e57d31b-8e6b-4007-8de1-431ae86a0f12,2006-11-07,562251||STOP
6e57d31b-8e6b-4007-8de1-431ae86a0f12,2007-02-24,313782||STOP
6e57d31b-8e6b-4007-8de1-431ae86a0f12,2007-06-10,313782||STOP


,code,desc
0,198405||START,Ibuprofen 100 MG Oral Tablet
1,308182||START,Amoxicillin 250 MG Oral Capsule
2,198405||START,Ibuprofen 100 MG Oral Tablet


,code,desc
38380,562251||STOP,Amoxicillin 250 MG / Clavulanate 125 MG Oral T...
38381,313782||STOP,Acetaminophen 325 MG Oral Tablet
38382,313782||STOP,Acetaminophen 325 MG Oral Tablet


In [ ]:
#export
def cleanup_img(imaging_studies, is_train):
    '''Clean `imaging` df'''
    imaging_studies.rename(columns={"DATE":"date", "PATIENT":"patient", "BODYSITE_CODE":"code", "BODYSITE_DESCRIPTION":"desc"}, inplace=True)

    if is_train: img_codes = imaging_studies.loc[:, ['code', 'desc']]
        
    imaging_studies = imaging_studies.loc[:, ['patient', 'date', 'code']]
    imaging_studies = imaging_studies.astype({'date':'datetime64'})
    imaging_studies.set_index('patient', inplace=True)
    return [imaging_studies, img_codes] if is_train else [imaging_studies]

In [ ]:
#export
def cleanup_procs(procedures, is_train):
    '''Clean `procedures` df'''
    procedures.rename(columns={"DATE":"date", "PATIENT":"patient", "CODE":"code", "DESCRIPTION":"desc"}, inplace=True)

    if is_train: proc_codes = procedures.loc[:, ['code', 'desc']]
    
    procedures = procedures.loc[:, ['patient', 'date', 'code']]
    procedures = procedures.astype({'date':'datetime64'})
    procedures.set_index('patient', inplace=True)
    return [procedures, proc_codes] if is_train else [procedures]

In [ ]:
#export
def cleanup_cnds(conditions, is_train):
    '''Clean `conditions` df'''
    conditions.drop(columns=['ENCOUNTER'], inplace=True)
    stops = pd.DataFrame(conditions.loc[conditions['STOP'].notnull(),:])
    conditions['CODE'] = conditions['CODE'].apply(lambda x: f'{str(x)}||START')
    stops['CODE'] = stops['CODE'].apply(lambda x: f'{str(x)}||STOP')
    conditions.drop(columns=['STOP'], inplace=True)
    stops.drop(columns=['START'], inplace=True)
    conditions.rename(columns={"START":"date", "PATIENT":"patient", "CODE":"code", "DESCRIPTION":"desc"}, inplace=True)
    stops.rename(columns={"STOP":"date", "PATIENT":"patient", "CODE":"code", "DESCRIPTION":"desc"}, inplace=True)
    conditions = conditions.append(stops, ignore_index=True)
        
    if is_train: cnd_codes = conditions.loc[:, ['code', 'desc']]
        
    conditions.drop(columns=['desc'], inplace=True)
    conditions = conditions.astype({'date':'datetime64'})
    conditions.set_index('patient', inplace=True)
    return [conditions, cnd_codes] if is_train else [conditions]

In [ ]:
#export
def cleanup_immns(immunizations, is_train):
    immunizations.rename(columns={"DATE":"date", "PATIENT":"patient", "CODE":"code", "DESCRIPTION":"desc"}, inplace=True)

    if is_train: imm_codes = immunizations.loc[:, ['code', 'desc']]
        
    immunizations = immunizations.loc[:, ['patient', 'date', 'code']]
    immunizations = immunizations.astype({'date':'datetime64'})
    immunizations.set_index('patient', inplace=True)
    return [immunizations, imm_codes] if is_train else [immunizations]

### Clean all

In [ ]:
#export
def cleanup_dataset(path, is_train, today=None):
    '''Clean all dfs in a split'''
    dfs = read_raw_ehrdata(path)
 
    pt_data   = cleanup_pts(dfs[0],   is_train, today)
    obs_data  = cleanup_obs(dfs[1],   is_train)
    alg_data  = cleanup_algs(dfs[2],  is_train)
    crpl_data = cleanup_crpls(dfs[3], is_train)
    med_data  = cleanup_meds(dfs[4],  is_train)
    img_data  = cleanup_img(dfs[5],   is_train)
    proc_data = cleanup_procs(dfs[6], is_train)
    cnd_data  = cleanup_cnds(dfs[7],  is_train)
    imm_data  = cleanup_immns(dfs[8], is_train)   
    
    data_tables = [pt_data[0], pt_data[1], obs_data[0], alg_data[0], crpl_data[0], med_data[0], img_data[0], proc_data[0], cnd_data[0], imm_data[0]]
    if is_train:
        code_tables = [pt_data[2], obs_data[1], alg_data[1], crpl_data[1], med_data[1], img_data[1], proc_data[1], cnd_data[1], imm_data[1]]
    
    return (data_tables, code_tables) if is_train else (data_tables)

In [ ]:
data_tables, code_tables = cleanup_dataset(f'{PATH_1K}/raw_split/train', is_train=True)

patients, pt_demographics, observations, allergies, \
careplans, medications, imaging_studies, procedures, conditions, immunizations = data_tables

pt_codes, obs_codes, alg_codes, crpl_codes, med_codes, img_codes, proc_codes, cnd_codes, imm_codes = code_tables

In [ ]:
conditions.count()

date    26426
code    26426
dtype: int64

In [ ]:
obs_codes.count()

orig_code    370128
desc         370128
value        370128
units        370128
type         370128
dtype: int64

## Extract Labels (y)

The labels we intend to predict are conditions and must be in the `CONDITIONS` dict
- Adding them to the `patients` df
- And adding the patient's age when the particular condition was recorded

In [ ]:
for key in CONDITIONS.keys():
    print(key,"::",CONDITIONS[key])

diabetes :: 44054006||START
stroke :: 230690007||START
alzheimers :: 26929004||START
coronary_heart :: 53741008||START


In [ ]:
#export
def extract_ys(patients, conditions, cnd_dict=CONDITIONS):
    '''Extract labels from conditions df and add them to patients df with age'''
    for key in cnd_dict.keys():
        patients = patients.merge(conditions[conditions.code==cnd_dict[key]], how='left', left_index=True, right_index=True)
        patients[f'{key}_y'] = patients.code.notna()
        patients[f'{key}_age'] = ((patients.date - patients.birthdate)//np.timedelta64(1,'Y'))
        patients = patients.drop(columns=['date','code'])
    return patients

In [ ]:
tmp_pts = extract_ys(patients, conditions, CONDITIONS)

In [ ]:
tmp_pts.count()

birthdate             664
diabetes_y            664
diabetes_age           37
stroke_y              664
stroke_age             42
alzheimers_y          664
alzheimers_age         18
coronary_heart_y      664
coronary_heart_age     34
dtype: int64

## Insert Age

Inserting patient's age in months and years into each record df
- this can be modified to records the patient's age in **days** or even **hours** that might be more relevant for datasets involving hospitalizations or ER admissions 

In [ ]:
#export
def insert_age(df, pts_df):
    '''Insert age in years and months into each of the rec dfs'''
    df = df.merge(pts_df, left_index=True, right_index=True)
    df['age']        = (df['date'] - df['birthdate'])//np.timedelta64(1,'Y')
    df['age_months'] = (df['date'] - df['birthdate'])//np.timedelta64(1,'M')
    return df.drop(columns=['date','birthdate'])

## Do-All Functions
The actual functions that will be called from other modules

In [ ]:
#export
def clean_raw_ehrdata(path, valid_pct=0.2, test_pct=0.2, today=None):
    '''Split, clean, preprocess & save raw EHR data'''
    split_ehr_dataset(path, valid_pct, test_pct)
    
    for split in ['train', 'valid', 'test']:
        split_path = f'{path}/raw_split/{split}'
        if split == 'train': data_tables, code_tables = cleanup_dataset(split_path, is_train=True, today=today)
        else               : data_tables = cleanup_dataset(split_path, is_train=False)
        patients, conditions, rec_tables = data_tables[0], data_tables[8], data_tables[2:]
        patients = extract_ys(patients, conditions, CONDITIONS)
        rec_dfs = [insert_age(rec_df, pd.DataFrame(patients.birthdate)) for rec_df in rec_tables]
        
        cleaned_dir = Path(f'{path}/cleaned/{split}')
        cleaned_dir.mkdir(parents=True, exist_ok=True)
    
        for rec_df,name in zip(rec_dfs,FILENAMES[1:]):
            rec_df.to_csv(f'{cleaned_dir}/{name}.csv')
        patients.reset_index(inplace=True)
        patients.to_csv(f'{cleaned_dir}/patients.csv', index_label='indx')
        data_tables[1].to_csv(f'{cleaned_dir}/patient_demographics.csv')
        print(f'Saved cleaned "{split}" data to {cleaned_dir}')
        
        if split == 'train':
            codes_dir = Path(f'{cleaned_dir}/codes')
            codes_dir.mkdir(parents=True, exist_ok=True)
            for code_df,name in zip(code_tables, FILENAMES):
                code_df.to_csv(f'{codes_dir}/code_{name}.csv', index_label='indx')
            print(f'Saved vocab code tables to {codes_dir}')

In [ ]:
#export
def load_cleaned_ehrdata(path):
    '''Load cleaned, age-filtered EHR data'''
    
    csv_names = FILENAMES.copy()
    csv_names.insert(1,'patient_demographics')
    
    train_dfs = [pd.read_csv(f'{path}/cleaned/train/{fname}.csv', low_memory=False, index_col=0) for fname in csv_names]
    valid_dfs = [pd.read_csv(f'{path}/cleaned/valid/{fname}.csv', low_memory=False, index_col=0) for fname in csv_names]
    test_dfs  = [pd.read_csv(f'{path}/cleaned/test/{fname}.csv', low_memory=False, index_col=0) for fname in csv_names]
                             
    return train_dfs, valid_dfs, test_dfs

In [ ]:
#export
def load_ehr_vocabcodes(path):
    '''Load codes for vocabs'''
    
    code_dfs = [pd.read_csv(f'{path}/cleaned/train/codes/code_{fname}.csv', low_memory=False, na_filter=False, index_col=0) for fname in FILENAMES]
                             
    return code_dfs

In [ ]:
train_dfs, valid_dfs, test_dfs = load_cleaned_ehrdata(PATH_1K)
code_dfs = load_ehr_vocabcodes(PATH_1K)

In [ ]:
# for df in train_dfs:
#     display(df.head())

In [ ]:
# for df in code_dfs:
#     display(df.head())

## Export -

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_setup.ipynb.
Converted 01_preprocessing_clean.ipynb.
Converted 02_preprocessing_vocab.ipynb.
Converted 03_preprocessing_transform.ipynb.
Converted 04_data.ipynb.
Converted 05_metrics.ipynb.
Converted 06_learn.ipynb.
Converted 07_models.ipynb.
Converted 08_experiment.ipynb.
Converted 99_quick_walkthru.ipynb.
Converted 99_running_exps.ipynb.
Converted index.ipynb.
